# Introduction

Pacti helps designers to reason about specifications and to manipulate them. These specifications are given to Pacti as assume-guarantee contracts, which are requirements of the form (assumptions, guarantees).

For Pacti, every contract has four elements:
- Input variables
- Output variables
- Assumptions
- Guarantees

The assumptions of a contract are only allowed to refer to the input variables. Guarantees are only allowed to refer to the input and output variables of the contract, i.e., no variables outside the interface of the contract may appear in the assumptions or guarantees.

The algebra of contracts has been extensively researched. This algebra allows us to compute new specifications from existing contracts. For example,
- The operation of composition allows us to compute the specification of the system formed by composing subsystems obeying the specifications being composed. In other words, composition is the system-building operation.
- The operation of quotient allows us to compute the specification of a component that needs to be composed with an existing subsystem so that the resulting system meets a top-level specification. Quotient is used to find missing-component specifications.
- The operation of merging is used to generate a single contract that simultaneously enforces the assumptions and guarantees of the contracts being merged. This is used to fuse contract viewpoints in concurrent design.

### Computing system specifications


Consider the following system:


![image](source/_static/exports/circuit_series_composition_background.svg)

Components $M$ and $M'$ obey, respectively, contracts $C = (|i| \le 2, o \le i \le 2o + 2)$ and $C' = (-1 \le o \le 1/5, o' \le o)$. We can use Pacti to obtain the specification of the system that comprises these two components as follows: 

In [ ]:
from pacti.terms.polyhedra import PolyhedralContract

contract1 = PolyhedralContract.from_string(
    InputVars=["i"],
    OutputVars=["o"],
    assumptions=["|i| <= 2"],
    guarantees=["o - i <= 0", "i - 2o <= 2"])

contract2 = PolyhedralContract.from_string(
    InputVars=["o"],
    OutputVars=["o_p"],
    assumptions=["o <= 0.2", "-o <= 1"],
    guarantees=["o_p - o <= 0"])

system_contract = contract1.compose(contract2)
print(system_contract)

Pacti gives us the specification of the system. Note that the resulting contract only involves the top-level inputs and outputs.


### System diagnostics

Suppose that we are trying to build a system as shown in the figure above with the same specification $C'$ for $M'$. Suppose that now the specification $C_n$ for $M$ is $(|i| <= 2, |o| <= 3)$.

In [ ]:
contract1_n = PolyhedralContract.from_string(
    InputVars=["i"],
    OutputVars=["o"],
    assumptions=["|i| <= 2"],
    guarantees=["|o| <= 3"])

contract_system = contract1_n.compose(contract2)

Pacti is unable to compute a system specification. In this case, this is due to the fact that our guarantee $|o| \le 3$ for $M$ does not satisfy the assumptions of $C'$.

### Quotient

Now consider the situation shown in the following diagram:

![image](source/_static/exports/circuit_series_quotient_background.png)

We wish to implement a system $M$ with specification $C = (|i| \le 1, o' = 2i + 1)$, and to do this we have available a component $M'$ with specification $C' = (|i| \le 2, o = 2i)$. We use the quotient operation in Pacti to obtain the specification of the component that we are missing--that would fill the question mark--so that the resulting object meets the specification $C$.

In [ ]:
contract_top_level = PolyhedralContract.from_string(
    InputVars=["i"],
    OutputVars=["o_p"],
    assumptions=["|i| <= 1"],
    guarantees=["o_p - 2i = 1"])

contract_existing_component = PolyhedralContract.from_string(
    InputVars=["i"],
    OutputVars=["o"],
    assumptions=["|i| <= 2"],
    guarantees=["o - 2i = 0"])

contract_missing_object = contract_top_level.quotient(contract_existing_component)
print(contract_missing_object)

Observe that Pacti tells us that the missing component's specification has input $o$ and output $o'$. The resulting specification is guaranteed to implement the top-level system when composed with the contract of the existing component. We can verify this:

In [ ]:
# compose quotient 
new_system_object = contract_missing_object.compose(contract_existing_component)
print(new_system_object.refines(contract_top_level))

### To learn more

This introductory examples show some of the analysis tasks we can carry out with Pacti. We have developed several case studies that discuss the application of Pacti in various design disciplines.